## To-do list
* Some cohort exploration based on deletion of subjects

* Look at variable correlations & clustering (if possible) -- suspect eg that systolic/diastolic will cluster based on prev work & means we can get away with using a variable score or one but not the other

* Create initial kitchen-sink model(s)
    * Identifier variables: ruid, visit_id, admit_date, discharge_date
    * Outcome variables: **readmit_30d**, readmit_time
    * Fixed effects (initially): ICD9, CPTs, medications, race, stay length, sex, DOB/age, pregnancy indicator
    * Random effects (initially): labs, number of transfers
    * Try as both: # transfers, stay length

* Model checking!!

In [7]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pymc3 as pm
sns.set()

from sklearn import preprocessing, model_selection
from sklearn.metrics import make_scorer, confusion_matrix, f1_score, roc_auc_score

In [2]:
df = pd.read_csv('../data/merged.csv')

In [3]:
df.head()

,ruid,visit_id,admit_date,discharge_date,stay_length,n_transfers,readmit_time,readmit_30d,sex,dob,...,gluc,k,na,pcv,plt-ct,systolic,diastolic,bmi,pregnancy_indicator,egfr
0,50135262,0,2007-02-08,2007-02-12,4 days 00:00:00.000000000,2,172 days 00:00:00.000000000,0,F,1949-09-20,...,138.0,4.40,136.0,32.0,334.0,140.0,58.0,44.710,0.0,123.677830
1,50135262,1,2007-08-03,2007-08-06,3 days 00:00:00.000000000,3,22 days 00:00:00.000000000,1,F,1949-09-20,...,213.5,4.45,135.0,39.0,291.5,121.0,61.0,45.025,0.0,89.505000
2,50135262,2,2007-08-28,2007-08-29,1 days 00:00:00.000000000,1,179 days 00:00:00.000000000,0,F,1949-09-20,...,266.0,3.90,139.0,38.0,308.0,131.0,60.0,46.230,0.0,107.450000
3,50135262,3,2008-02-24,2008-02-28,4 days 00:00:00.000000000,2,44 days 00:00:00.000000000,0,F,1949-09-20,...,230.0,4.00,135.0,38.0,274.0,151.0,74.0,47.140,0.0,73.010770
4,50135262,4,2008-04-12,2008-04-13,1 days 00:00:00.000000000,1,928 days 00:00:00.000000000,0,F,1949-09-20,...,162.0,4.10,139.0,36.0,330.0,134.0,66.0,47.360,0.0,84.358415


In [5]:
df.shape

(21041, 66)

In [8]:
df.columns

Index(['ruid', 'visit_id', 'admit_date', 'discharge_date', 'stay_length',
       'n_transfers', 'readmit_time', 'readmit_30d', 'sex', 'dob', 'dod',
       'race', 'cpt_anesthesia', 'cpt_eval_manage', 'cpt_expired',
       'cpt_medicine', 'cpt_modifier', 'cpt_path_lab', 'cpt_radiology',
       'cpt_surgery', 'cpt_unknown', 'icd_dx_blood', 'icd_dx_circulatory',
       'icd_dx_congenital', 'icd_dx_digestive', 'icd_dx_endocrine',
       'icd_dx_external', 'icd_dx_gu', 'icd_dx_infection', 'icd_dx_injury',
       'icd_dx_mental', 'icd_dx_muscskel', 'icd_dx_neoplasm', 'icd_dx_nervous',
       'icd_dx_obstetric', 'icd_dx_perinatal', 'icd_dx_respiratory',
       'icd_dx_skin', 'icd_dx_symptoms', 'icd_proc', 'icd_visit',
       'med_Antihypertensive Agents', 'med_Analgesics, Opioid',
       'med_Narcotics', 'med_Antipyretics', 'med_Anti-Bacterial Agents',
       'med_Anti-Inflammatory Agents, Non-Steroidal',
       'med_Analgesics, Non-Narcotic', 'med_Antiemetics', 'med_Diuretics',
       'med_A

In [9]:
.03*21000

630.0